In [ ]:
import requests
import datetime
import plotly.graph_objs as go
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import numpy as np

# API Configuration
API_KEY = "0dcac01ceb5b8e273abc1095c5910a20"
BASE_URL = "https://api.openweathermap.org/data/2.5/forecast"

# Dash App Initialization
app = dash.Dash(__name__)
app.title = "Geo Weather Analytics"

# Modern Layout
app.layout = html.Div(
    style={
        'backgroundColor': '#0F1419',
        'color': '#FFFFFF',
        'fontFamily': 'Poppins, sans-serif',
        'padding': '20px',
        'minHeight': '100vh'
    },
    children=[
        html.H1(
            "Geo Weather Analytics",
            style={
                'textAlign': 'center',
                'fontSize': '28px',
                'fontWeight': '600',
                'marginBottom': '20px',
                'color': '#00BFFF',
                'textShadow': '0 0 8px rgba(0,191,255,0.6)'
            }
        ),
        html.Div(
            [
                dcc.Input(
                    id='city-input',
                    type='text',
                    value='London',
                    placeholder='Enter City Name',
                    style={
                        'width': '180px',
                        'padding': '8px',
                        'borderRadius': '6px',
                        'border': 'none',
                        'marginRight': '10px',
                        'backgroundColor': '#1F2A44',
                        'color': '#FFF',
                        'fontSize': '14px'
                    }
                ),
                html.Button(
                    "Choose Location",
                    id='update-button',
                    n_clicks=0,
                    style={
                        'padding': '4px 8px',
                        'borderRadius': '6px',
                        'background': 'linear-gradient(90deg, #1E90FF, #00CED1)',
                        'border': 'none',
                        'color': '#FFF',
                        'fontSize': '14px',
                        'cursor': 'pointer'
                    }
                ),
                html.Div(id='error-message', style={'color': '#FF4500', 'marginTop': '8px', 'fontSize': '12px'})
            ],
            style={'textAlign': 'center', 'marginBottom': '30px'}
        ),
        html.Div(id='kpi-container', style={'display': 'flex', 'justifyContent': 'space-between', 'marginBottom': '30px', 'flexWrap': 'wrap'}),
        html.Div(
            [
                dcc.Graph(id='temp-feels-area', style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top'}),
                dcc.Graph(id='humidity-cloud-heatmap', style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top'}),
                dcc.Graph(id='rain-gauge', style={'width': '24%', 'display': 'inline-block', 'verticalAlign': 'top'}),
                dcc.Graph(id='pop-gauge', style={'width': '24%', 'display': 'inline-block', 'verticalAlign': 'top'}),
                dcc.Graph(id='wind-rose', style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),
                dcc.Graph(id='visibility-dew-bar', style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top'}),
                dcc.Graph(id='uvi-gdd-bar', style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top'}),
            ],
            style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'center'}
        ),
        dcc.Interval(id='interval-update', interval=600000, n_intervals=0)
    ]
)

# Function to fetch weather data
def fetch_weather_data(city):
    params = {"q": city, "appid": API_KEY, "units": "metric"}
    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        if response.status_code == 200:
            return response.json()
        return None
    except requests.RequestException:
        return None

# Callback for updating the dashboard
@app.callback(
    [Output('temp-feels-area', 'figure'),
     Output('humidity-cloud-heatmap', 'figure'),
     Output('rain-gauge', 'figure'),
     Output('pop-gauge', 'figure'),
     Output('wind-rose', 'figure'),
     Output('visibility-dew-bar', 'figure'),
     Output('uvi-gdd-bar', 'figure'),
     Output('kpi-container', 'children'),
     Output('error-message', 'children')],
    [Input('interval-update', 'n_intervals'),
     Input('update-button', 'n_clicks')],
    [dash.dependencies.State('city-input', 'value')]
)
def update_dashboard(n_intervals, n_clicks, city):
    data = fetch_weather_data(city)
    if not data or 'list' not in data:
        return (dash.no_update,) * 9 + (f"Error: Unable to fetch data for '{city}'. Check city name or API status.",)

    df = pd.DataFrame(data['list'])
    df['datetime'] = pd.to_datetime(df['dt'], unit='s')
    now = datetime.datetime.now()
    df = df[(df['datetime'] >= now) & (df['datetime'] <= now + datetime.timedelta(days=5))]

    if df.empty:
        return (dash.no_update,) * 9 + (f"No forecast data for '{city}' for the next 5 days.",)

    # Extract metrics
    df['temp'] = df['main'].apply(lambda x: x['temp'])
    df['feels_like'] = df['main'].apply(lambda x: x['feels_like'])
    df['temp_min'] = df['main'].apply(lambda x: x['temp_min'])
    df['temp_max'] = df['main'].apply(lambda x: x['temp_max'])
    df['humidity'] = df['main'].apply(lambda x: x['humidity'])
    df['clouds'] = df['clouds'].apply(lambda x: x['all'])
    df['rain'] = df['rain'].apply(lambda x: x.get('3h', 0) if isinstance(x, dict) else 0)
    df['wind_speed'] = df['wind'].apply(lambda x: x['speed'])
    df['wind_gust'] = df['wind'].apply(lambda x: x.get('gust', 0))
    df['wind_dir'] = df['wind'].apply(lambda x: x.get('deg', 0))
    df['visibility'] = df['visibility'] / 1000
    df['pop'] = df['pop'] * 100

    # Calculated Metrics
    df['uvi'] = 10 * (1 - df['clouds'] / 100)  # Approximate UVI
    df['gdd'] = np.maximum((df['temp_max'] + df['temp_min']) / 2 - 10, 0)
    df['wind_chill'] = np.where(
        (df['temp'] <= 10) & (df['wind_speed'] > 3),
        13.12 + 0.6215 * df['temp'] - 11.37 * (df['wind_speed'] ** 0.16) + 0.3965 * df['temp'] * (df['wind_speed'] ** 0.16),
        df['temp']
    )

    # Ensure non-zero values for bar charts to make bars visible
    df['rain'] = df['rain'].replace(0, 0.001)  # Small value to ensure bars render
    df['visibility'] = df['visibility'].replace(0, 0.001)
    df['uvi'] = df['uvi'].replace(0, 0.001)
    df['gdd'] = df['gdd'].replace(0, 0.001)

    x_range = [df['datetime'].min(), df['datetime'].max()]

    # 1. Temperature & Feels Like (Left)
    fig_temp_feels = go.Figure()
    fig_temp_feels.add_trace(go.Scatter(
        x=df['datetime'], y=df['temp'], mode='lines', stackgroup='one',
        line=dict(width=0), fillcolor='rgba(0,191,255,0.5)', name='Temperature (°C)'
    ))
    fig_temp_feels.add_trace(go.Scatter(
        x=df['datetime'], y=df['feels_like'], mode='lines', stackgroup='one',
        line=dict(width=0), fillcolor='rgba(255,105,180,0.5)', name='Feels Like (°C)'
    ))
    fig_temp_feels.update_layout(
        title=dict(text="Temperature & Feels Like (Next 5 Days)", font=dict(color='#00BFFF', size=16, weight='bold'), y=0.96, x=0.45, xanchor='center', yanchor='top'),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), showlegend=True, xaxis_range=x_range, yaxis=dict(showgrid=False), xaxis=dict(showgrid=False),
        margin=dict(t=40), height=250,
          # Legend settings
        legend=dict(
            orientation="h",  # Horizontal orientation
            x=0.48,  # Center horizontally
            y=0.93,  # Place above the title (adjust as needed)
            xanchor='center',  # Anchor the legend's x position at its center
            yanchor='bottom',  # Anchor the legend's y position at its bottom
            font=dict(size=12, color='#FFF'),  # Match font color with the theme
            bgcolor='rgba(0,0,0,0)',  # Transparent background for the legend
            bordercolor='rgba(0,0,0,0)',  # No border
        )
    )

    # 2. Humidity & Cloudiness (Right)
    fig_humidity_cloud = go.Figure(data=go.Heatmap(
        z=[df['humidity'], df['clouds']],
        x=df['datetime'],
        y=['Humidity (%)', 'Cloudiness (%)'],
        colorscale='Viridis',
        hovertemplate='%{y}: %{z:.1f}% at %{x|%Y-%m-%d %H:%M}<extra></extra>'
    ))
    fig_humidity_cloud.update_layout(
        title=dict(text="Humidity & Cloudiness Heatmap (Next 5 Days)", font=dict(color='#00BFFF', size=16, weight='bold'), y=0.95, x=0.5, xanchor='center', yanchor='top'),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), xaxis_range=x_range, xaxis=dict(showgrid=False), yaxis=dict(showgrid=False),
        margin=dict(t=40), height=250
    )

    """# 3. Rainfall (Bar)
    fig_rain = go.Figure(data=go.Bar(
        x=df['datetime'], y=df['rain'], marker_color='#00CED1', opacity=0.8
    ))
    fig_rain.update_layout(
        title=dict(text="Rainfall (Next 5 Days)", font=dict(color='#00BFFF', size=16, weight='bold'), y=0.95, x=0.5, xanchor='center', yanchor='top'),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), yaxis_title="Rainfall (mm)", xaxis=dict(showgrid=False), yaxis=dict(showgrid=False),
        margin=dict(t=40), height=400
    )"""

  # 3. Rainfall
    fig_rain = go.Figure(go.Indicator(
        mode="gauge+number",
        value=df['rain'].mean(),
        domain={'x': [0, 1], 'y': [0, 1]},
        #title={'text': "Avg Rainfall (mm/3h)"},
        gauge={
            'axis': {'range': [0, 10]},
            'bar': {'color': "#00BFFF"},
            'steps': [{'range': [0, 2], 'color': "lightgray"}, {'range': [2, 5], 'color': "gray"}, {'range': [5, 10], 'color': "darkgray"}],
            'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 8}
        }
    ))
    fig_rain.update_layout(
         title=dict(text=" Avg Rainfall (mm/3h) (Next 5 Days)", font=dict(color='#00BFFF', size=16, weight='bold'), y=0.95, x=0.5, xanchor='center', yanchor='top'),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), yaxis_title="Rainfall (mm)", xaxis=dict(showgrid=False), yaxis=dict(showgrid=False),
        margin=dict(t=40), height=400
    )
    """# 4. Precipitation Probability (Gauge)
    avg_pop = df['pop'].mean()
    fig_pop = go.Figure(go.Indicator(
        mode="gauge+number", value=avg_pop, domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "Probability (%)"},
        gauge={'axis': {'range': [0, 100]}, 'bar': {'color': "#4682B4"}, 'steps': [
            {'range': [0, 50], 'color': "#4682B4"}, {'range': [50, 100], 'color': "#FF4500"}]}
    ))
    fig_pop.update_layout(
        title=dict(text="Precipitation (Next 5 Days)", font=dict(color='#00BFFF', size=16, weight='bold'), y=0.95, x=0.5, xanchor='center', yanchor='top'),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), height=300
    )"""

    # 4. Probability of Precipitation
    fig_pop = go.Figure(go.Indicator(
        mode="gauge+number",
        value=df['pop'].mean(),
        domain={'x': [0, 1], 'y': [0, 1]},
       # title={'text': "Avg Precipitation Probability (%)"},
        gauge={
            'axis': {'range': [0, 100]},
            'bar': {'color': "#00CED1"},
            'steps': [{'range': [0, 30], 'color': "lightgray"}, {'range': [30, 70], 'color': "gray"}, {'range': [70, 100], 'color': "darkgray"}],
            'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 90}
        }
    ))
    fig_pop.update_layout(
       title=dict(text="Avg Precipitation Probability (%) (Next 5 Days)", font=dict(color='#00BFFF', size=16, weight='bold'), y=0.95, x=0.5, xanchor='center', yanchor='top'),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), height=370
    )


# 4. Wind Rose Diagram (Combining Direction, Speed, and Gusts)
    # Bin wind directions into 8 sectors (every 45 degrees)
    bins = np.arange(0, 360 + 45, 45)
    labels = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
    df['wind_dir_bin'] = pd.cut(df['wind_dir'], bins=bins, labels=labels, include_lowest=True, right=False)
    
    # Group by direction and calculate average speed and gusts
    wind_data = df.groupby('wind_dir_bin').agg({'wind_speed': 'mean', 'wind_gust': 'mean'}).reset_index()
    wind_data['wind_dir_bin'] = pd.Categorical(wind_data['wind_dir_bin'], categories=labels, ordered=True)
    
    # Create wind rose
    fig_wind_rose = go.Figure()
    fig_wind_rose.add_trace(go.Barpolar(
        r=wind_data['wind_speed'],
        theta=[0, 45, 90, 135, 180, 225, 270, 315],
        marker_color=wind_data['wind_gust'],
        marker_colorscale='Viridis',
        opacity=0.8,
        name='Wind Speed (m/s)',
        hovertemplate='Direction: %{theta}°<br>Speed: %{r:.1f} m/s<br>Gust: %{marker.color:.1f} m/s<extra></extra>'
    ))
    fig_wind_rose.update_layout(
        title=dict(
            text="Wind Rose (Next 5 Days)",
            font=dict(color='#00BFFF', size=16, family='Poppins, sans-serif', weight='bold'),
            y=0.95,
            x=0.43,
            xanchor='center',
            yanchor='top'
        ),
        template='plotly_dark',
        polar=dict(
            radialaxis=dict(visible=True, range=[0, max(wind_data['wind_speed'].max(), 5)], showticklabels=True),
            angularaxis=dict(direction="clockwise", rotation=90, tickvals=[0, 45, 90, 135, 180, 225, 270, 315], ticktext=['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'])
        ),
        showlegend=True,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'),
        margin=dict(t=60))



    # 6. Visibility & Wind Chill
    fig_vis_dew = go.Figure()
    fig_vis_dew.add_trace(go.Bar(
        x=df['datetime'], y=df['visibility'], marker_color='#FFD700', opacity=0.7,
        name='Visibility (km)'  # Ensure the trace has a name for the legend
    ))
    fig_vis_dew.add_trace(go.Scatter(
        x=df['datetime'], y=df['wind_chill'], mode='lines', line=dict(color='#FF69B4', width=1),
        name='Wind Chill (°C)', yaxis='y2'
    ))
    fig_vis_dew.update_layout(
        title=dict(text="Visibility & Wind Chill (5 Days)", font=dict(color='#00BFFF', size=16, weight='bold'), y=1.0, x=0.5, xanchor='center', yanchor='top'),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), yaxis_title="Visibility (km)",
        yaxis2=dict(overlaying='y', side='right', title="Wind Chill (°C)", showgrid=False),
        xaxis=dict(showgrid=False), yaxis=dict(showgrid=False),
        margin=dict(t=40), height=250,
        # Legend settings
        legend=dict(
            orientation="h",  # Horizontal orientation
            x=0.5,  # Center horizontally
            y=0.95,  # Place above the title (adjust as needed)
            xanchor='center',  # Anchor the legend's x position at its center
            yanchor='bottom',  # Anchor the legend's y position at its bottom
            font=dict(size=12, color='#FFF'),  # Match font color with the theme
            bgcolor='rgba(0,0,0,0)',  # Transparent background for the legend
            bordercolor='rgba(0,0,0,0)',  # No border
        )
    )



    # 7. UV Index & GDD
    fig_uvi_gdd = go.Figure()
    fig_uvi_gdd.add_trace(go.Bar(
        x=df['datetime'], y=df['uvi'], marker_color='#FFA500', name='UV Index', opacity=0.7
    ))
    fig_uvi_gdd.add_trace(go.Bar(
        x=df['datetime'], y=df['gdd'].cumsum(), marker_color='#32CD32', name='Cumulative GDD', opacity=0.7
    ))
    fig_uvi_gdd.update_layout(
        title=dict(text="UV Index & GDD (Next 5 Days)", font=dict(color='#00BFFF', size=16, weight='bold'), y=1.0, x=0.48, xanchor='center', yanchor='top'),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), yaxis_title="UV Index / GDD",
        xaxis=dict(showgrid=False), yaxis=dict(showgrid=False),
        margin=dict(t=40), height=250,
         # Legend settings
        legend=dict(
            orientation="h",  # Horizontal orientation
            x=0.48,  # Center horizontally
            y=0.95,  # Place above the title (adjust as needed)
            xanchor='center',  # Anchor the legend's x position at its center
            yanchor='bottom',  # Anchor the legend's y position at its bottom
            font=dict(size=12, color='#FFF'),  # Match font color with the theme
            bgcolor='rgba(0,0,0,0)',  # Transparent background for the legend
            bordercolor='rgba(0,0,0,0)',  # No border
        )
    )

    # KPIs
    wind_dir_counts = df['wind_dir_bin'].value_counts()
    kpis = [
        html.Div(f"Avg Temp: {df['temp'].mean():.1f}°C", style={'background': 'linear-gradient(90deg, #00FFFF, #4682B4)', 'padding': '5px', 'borderRadius': '5px', 'width': 'auto', 'textAlign': 'center', 'margin': '0px', 'fontSize': '12px'}),
        html.Div(f"Total Rain: {df['rain'].sum():.2f} mm", style={'background': 'linear-gradient(90deg, #00CED1, #20B2AA)', 'padding': '5px', 'borderRadius': '5px', 'width': 'auto', 'textAlign': 'center', 'margin': '0px', 'fontSize': '12px'}),
        html.Div(f"Max Gust: {df['wind_gust'].max():.1f} m/s", style={'background': 'linear-gradient(90deg, #FF4500, #FF6347)', 'padding': '5px', 'borderRadius': '5px', 'width': 'auto', 'textAlign': 'center', 'margin': '0px', 'fontSize': '12px'}),
        html.Div(f"Cum GDD: {df['gdd'].sum():.1f}", style={'background': 'linear-gradient(90deg, #32CD32, #228B22)', 'padding': '5px', 'borderRadius': '5px', 'width': 'auto', 'textAlign': 'center', 'margin': '0px', 'fontSize': '12px'}),
        html.Div(f"Prev Wind: {wind_dir_counts.idxmax()}", style={'background': 'linear-gradient(90deg, #FF69B4, #FFB6C1)', 'padding': '5px', 'borderRadius': '5px', 'width': 'auto', 'textAlign': 'center', 'margin': '0px', 'fontSize': '12px'}),
    ]
   


    return fig_temp_feels, fig_humidity_cloud, fig_rain, fig_pop, fig_wind_rose, fig_vis_dew, fig_uvi_gdd, kpis, ""

if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=8050)

C:\Users\User\AppData\Local\Temp\ipykernel_3220\2031524488.py:276: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\User\AppData\Local\Temp\ipykernel_3220\2031524488.py:276: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\User\AppData\Local\Temp\ipykernel_3220\2031524488.py:276: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\User\AppData\Local\Temp\ipykernel_3220\2031524488.py:276: FutureWarning:

Th

In [ ]:
import requests
import datetime
import plotly.graph_objs as go
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import numpy as np

# API Configuration
API_KEY = "0dcac01ceb5b8e273abc1095c5910a20"
BASE_URL = "https://api.openweathermap.org/data/2.5/forecast"

# Dash App Initialization
app = dash.Dash(__name__)
app.title = "Geo Weather Analytics"

# Modern Layout
app.layout = html.Div(
    style={
        'backgroundColor': '#0F1419',
        'color': '#FFFFFF',
        'fontFamily': 'Poppins, sans-serif',
        'padding': '50px',
        'minHeight': '100vh'
    },
    children=[
        html.H1(
            "Geo Weather Analytics",
            style={
                'textAlign': 'center',
                'fontSize': '42px',
                'fontWeight': '600',
                'marginBottom': '40px',
                'color': '#00BFFF',
                'textShadow': '0 0 10px rgba(0,191,255,0.8)'  # Glittering effect
            }
        ),
        html.Div(
            [
                dcc.Input(
                    id='city-input',
                    type='text',
                    value='London',
                    placeholder='Enter City Name',
                    style={
                        'width': '280px',
                        'padding': '15px',
                        'borderRadius': '10px',
                        'border': 'none',
                        'marginRight': '20px',
                        'backgroundColor': '#1F2A44',
                        'color': '#FFF',
                        'fontSize': '18px',
                        'boxShadow': '0 4px 6px rgba(0,0,0,0.1)'
                    }
                ),
                html.Button(
                    "Update",
                    id='update-button',
                    n_clicks=0,
                    style={
                        'padding': '15px 30px',
                        'borderRadius': '10px',
                        'background': 'linear-gradient(90deg, #1E90FF, #00CED1)',
                        'border': 'none',
                        'color': '#FFF',
                        'fontSize': '18px',
                        'cursor': 'pointer',
                        'boxShadow': '0 4px 6px rgba(0,0,0,0.2)',
                        'transition': 'transform 0.2s',
                        ':hover': {'transform': 'scale(1.05)'}
                    }
                ),
                html.Div(id='error-message', style={'color': '#FF4500', 'marginTop': '20px', 'fontSize': '16px'})
            ],
            style={'textAlign': 'center', 'marginBottom': '50px'}
        ),
        html.Div(id='kpi-container', style={'display': 'flex', 'justifyContent': 'space-around', 'marginBottom': '50px', 'flexWrap': 'wrap'}),
        html.Div(
            [
                dcc.Graph(id='temp-feels-area', style={'width': '100%', 'marginBottom': '30px'}),
                dcc.Graph(id='humidity-cloud-heatmap', style={'width': '100%', 'marginBottom': '30px'}),
                dcc.Graph(id='rain-pop-gauge', style={'width': '48%', 'display': 'inline-block'}),
                dcc.Graph(id='wind-rose', style={'width': '48%', 'display': 'inline-block'}),
                dcc.Graph(id='visibility-dew-bar', style={'width': '100%', 'marginTop': '30px'}),
            ],
            style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'space-between'}
        ),
        dcc.Interval(id='interval-update', interval=600000, n_intervals=0)  # Update every 10 minutes
    ]
)

# Function to fetch weather data
def fetch_weather_data(city):
    params = {"q": city, "appid": API_KEY, "units": "metric"}
    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        if response.status_code == 200:
            return response.json()
        return None
    except requests.RequestException:
        return None

# Callback for updating the dashboard
@app.callback(
    [Output('temp-feels-area', 'figure'),
     Output('humidity-cloud-heatmap', 'figure'),
     Output('rain-pop-gauge', 'figure'),
     Output('wind-rose', 'figure'),
     Output('visibility-dew-bar', 'figure'),
     Output('kpi-container', 'children'),
     Output('error-message', 'children')],
    [Input('interval-update', 'n_intervals'),
     Input('update-button', 'n_clicks')],
    [dash.dependencies.State('city-input', 'value')]
)
def update_dashboard(n_intervals, n_clicks, city):
    data = fetch_weather_data(city)
    if not data or 'list' not in data:
        return (dash.no_update, dash.no_update, dash.no_update, dash.no_update, dash.no_update, dash.no_update,
                f"Error: Unable to fetch data for '{city}'. Check city name or API status.")

    df = pd.DataFrame(data['list'])
    df['datetime'] = pd.to_datetime(df['dt'], unit='s')
    now = datetime.datetime.now()
    df = df[(df['datetime'] >= now) & (df['datetime'] <= now + datetime.timedelta(days=5))]

    if df.empty:
        return (dash.no_update, dash.no_update, dash.no_update, dash.no_update, dash.no_update, dash.no_update,
                f"No forecast data available for '{city}' for the next 5 days.")

    # Extract metrics
    df['temp'] = df['main'].apply(lambda x: x['temp'])
    df['feels_like'] = df['main'].apply(lambda x: x['feels_like'])
    df['humidity'] = df['main'].apply(lambda x: x['humidity'])
    df['clouds'] = df['clouds'].apply(lambda x: x['all'])
    df['rain'] = df['rain'].apply(lambda x: x.get('3h', 0) if isinstance(x, dict) else 0)
    df['wind_speed'] = df['wind'].apply(lambda x: x['speed'])
    df['wind_gust'] = df['wind'].apply(lambda x: x.get('gust', 0))
    df['wind_dir'] = df['wind'].apply(lambda x: x.get('deg', 0))
    df['visibility'] = df['visibility'] / 1000
    df['dew_point'] = df['main'].apply(lambda x: x.get('dew_point', 0))
    df['pop'] = df['pop'] * 100

    x_range = [df['datetime'].min(), df['datetime'].max()]

    # 1. Temperature & Feels Like (Stacked Area Chart)
    fig_temp_feels = go.Figure()
    fig_temp_feels.add_trace(go.Scatter(
        x=df['datetime'], y=df['temp'], mode='lines', stackgroup='one',
        line=dict(width=0), fillcolor='rgba(0,255,255,0.5)', name='Temperature (°C)'
    ))
    fig_temp_feels.add_trace(go.Scatter(
        x=df['datetime'], y=df['feels_like'], mode='lines', stackgroup='one',
        line=dict(width=0), fillcolor='rgba(255,105,180,0.5)', name='Feels Like (°C)'
    ))
    fig_temp_feels.update_layout(
        title=dict(
            text="Temperature & Feels Like (Next 5 Days)",
            font=dict(color='#00BFFF', size=20, family='Poppins, sans-serif', weight='bold'),
            y=0.95,  # Adjust title position
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), showlegend=True, xaxis_range=x_range,
        yaxis=dict(showgrid=False, zeroline=False), xaxis=dict(showgrid=False),
        margin=dict(t=60)  # Add margin to prevent overlap
    )

    # 2. Humidity & Cloudiness (Heatmap)
    fig_humidity_cloud = go.Figure(data=go.Heatmap(
        z=[df['humidity'], df['clouds']],
        x=df['datetime'],
        y=['Humidity (%)', 'Cloudiness (%)'],
        colorscale='Viridis',
        hovertemplate='%{y}: %{z:.1f}% at %{x|%Y-%m-%d %H:%M}<extra></extra>'
    ))
    fig_humidity_cloud.update_layout(
        title=dict(
            text="Humidity & Cloudiness Heatmap (Next 5 Days)",
            font=dict(color='#00BFFF', size=20, family='Poppins, sans-serif', weight='bold'),
            y=0.95,
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), xaxis_range=x_range, xaxis=dict(showgrid=False), yaxis=dict(showgrid=False),
        margin=dict(t=60)
    )

    # 3. Rain & Probability of Precipitation (Gauge + Bar)
    avg_pop = df['pop'].mean()
    fig_rain_pop = go.Figure()
    fig_rain_pop.add_trace(go.Indicator(
        mode="gauge+number", value=avg_pop, domain={'x': [0, 1], 'y': [0.5, 1]},
        title={'text': "Precipitation Probability (%)"},
        gauge={'axis': {'range': [0, 100]}, 'bar': {'color': "#00CED1"}, 'steps': [
            {'range': [0, 50], 'color': "#4682B4"}, {'range': [50, 100], 'color': "#FF4500"}]}
    ))
    fig_rain_pop.add_trace(go.Bar(
        x=df['datetime'], y=df['rain'], marker_color='#00CED1', name='Rainfall (mm)',
        yaxis='y2', opacity=0.7
    ))
    fig_rain_pop.update_layout(
        title=dict(
            text="Rainfall & Precipitation Chance (Next 5 Days)",
            font=dict(color='#00BFFF', size=20, family='Poppins, sans-serif', weight='bold'),
            y=0.95,
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), height=400,
        yaxis2=dict(overlaying='y', side='right', showgrid=False, title="Rainfall (mm)"),
        xaxis=dict(showgrid=False), yaxis=dict(showgrid=False),
        margin=dict(t=60)
    )

    # 4. Wind Rose Diagram (Combining Direction, Speed, and Gusts)
    # Bin wind directions into 8 sectors (every 45 degrees)
    bins = np.arange(0, 360 + 45, 45)
    labels = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
    df['wind_dir_bin'] = pd.cut(df['wind_dir'], bins=bins, labels=labels, include_lowest=True, right=False)
    
    # Group by direction and calculate average speed and gusts
    wind_data = df.groupby('wind_dir_bin').agg({'wind_speed': 'mean', 'wind_gust': 'mean'}).reset_index()
    wind_data['wind_dir_bin'] = pd.Categorical(wind_data['wind_dir_bin'], categories=labels, ordered=True)
    
    # Create wind rose
    fig_wind_rose = go.Figure()
    fig_wind_rose.add_trace(go.Barpolar(
        r=wind_data['wind_speed'],
        theta=[0, 45, 90, 135, 180, 225, 270, 315],
        marker_color=wind_data['wind_gust'],
        marker_colorscale='Viridis',
        opacity=0.8,
        name='Wind Speed (m/s)',
        hovertemplate='Direction: %{theta}°<br>Speed: %{r:.1f} m/s<br>Gust: %{marker.color:.1f} m/s<extra></extra>'
    ))
    fig_wind_rose.update_layout(
        title=dict(
            text="Wind Rose (Next 5 Days)",
            font=dict(color='#00BFFF', size=20, family='Poppins, sans-serif', weight='bold'),
            y=0.95,
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
        template='plotly_dark',
        polar=dict(
            radialaxis=dict(visible=True, range=[0, max(wind_data['wind_speed'].max(), 5)], showticklabels=True),
            angularaxis=dict(direction="clockwise", rotation=90, tickvals=[0, 45, 90, 135, 180, 225, 270, 315], ticktext=['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'])
        ),
        showlegend=True,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'),
        margin=dict(t=60)
    )

    # 5. Visibility & Dew Point (Bar Chart)
    fig_vis_dew = go.Figure()
    fig_vis_dew.add_trace(go.Bar(
        x=df['datetime'], y=df['visibility'], marker_color='#FFD700', name='Visibility (km)', opacity=0.7
    ))
    fig_vis_dew.add_trace(go.Scatter(
        x=df['datetime'], y=df['dew_point'], mode='lines', line=dict(color='#FF69B4', width=3), name='Dew Point (°C)', yaxis='y2'
    ))
    fig_vis_dew.update_layout(
        title=dict(
            text="Visibility & Dew Point (Next 5 Days)",
            font=dict(color='#00BFFF', size=20, family='Poppins, sans-serif', weight='bold'),
            y=0.95,
            x=0.5,
            xanchor='center',
            yanchor='top'
        ),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), yaxis_title="Visibility (km)",
        yaxis2=dict(overlaying='y', side='right', title="Dew Point (°C)", showgrid=False),
        xaxis=dict(showgrid=False), yaxis=dict(showgrid=False),
        margin=dict(t=60)
    )

    # KPIs
    wind_dir_counts = df['wind_dir_bin'].value_counts()
    kpis = [
        html.Div(f"Avg Temp: {df['temp'].mean():.1f}°C", style={'background': 'linear-gradient(90deg, #00FFFF, #4682B4)', 'padding': '15px', 'borderRadius': '8px', 'width': '18%', 'textAlign': 'center', 'margin': '5px'}),
        html.Div(f"Total Rain: {df['rain'].sum():.2f} mm", style={'background': 'linear-gradient(90deg, #00CED1, #20B2AA)', 'padding': '15px', 'borderRadius': '8px', 'width': '18%', 'textAlign': 'center', 'margin': '5px'}),
        html.Div(f"Max Gust: {df['wind_gust'].max():.1f} m/s", style={'background': 'linear-gradient(90deg, #FF4500, #FF6347)', 'padding': '15px', 'borderRadius': '8px', 'width': '18%', 'textAlign': 'center', 'margin': '5px'}),
        html.Div(f"Avg Visibility: {df['visibility'].mean():.1f} km", style={'background': 'linear-gradient(90deg, #FFD700, #FFA500)', 'padding': '15px', 'borderRadius': '8px', 'width': '18%', 'textAlign': 'center', 'margin': '5px'}),
        html.Div(f"Prevailing Wind: {wind_dir_counts.idxmax()}", style={'background': 'linear-gradient(90deg, #FF69B4, #FFB6C1)', 'padding': '15px', 'borderRadius': '8px', 'width': '18%', 'textAlign': 'center', 'margin': '5px'}),
    ]

    return fig_temp_feels, fig_humidity_cloud, fig_rain_pop, fig_wind_rose, fig_vis_dew, kpis, ""

if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=8050)

C:\Users\User\AppData\Local\Temp\ipykernel_3220\401876946.py:231: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\User\AppData\Local\Temp\ipykernel_3220\401876946.py:231: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\User\AppData\Local\Temp\ipykernel_3220\401876946.py:231: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\User\AppData\Local\Temp\ipykernel_3220\401876946.py:231: FutureWarning:

The de

In [41]:
import requests
import datetime
import plotly.graph_objs as go
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import numpy as np
from opcua import Client, ua, Server
from threading import Thread
from collections import deque
import time
import logging
import signal
import socket

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logging.getLogger("opcua").setLevel(logging.WARNING)

# Function to find an available port
def find_free_port(start_port=4842):
    port = start_port
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(('localhost', port))
                return port
            except OSError:
                port += 1

# OPC UA Configuration
port = find_free_port()
OPC_SERVER_URL = f"opc.tcp://localhost:{port}"
weather_data = deque(maxlen=40)
server_running = True

# Dash App Initialization
app = dash.Dash(__name__)
app.title = "Geo Weather Analytics"

# Layout
app.layout = html.Div(
    style={
        'backgroundColor': '#0F1419',
        'color': '#FFFFFF',
        'fontFamily': 'Poppins, sans-serif',
        'padding': '20px',
        'minHeight': '100vh'
    },
    children=[
        html.H1("Geo Weather Analytics", style={'textAlign': 'center', 'fontSize': '28px', 'fontWeight': '600', 'marginBottom': '20px', 'color': '#00BFFF', 'textShadow': '0 0 8px rgba(0,191,255,0.6)'}),
        html.Div([
            dcc.Input(id='city-input', type='text', value='London', placeholder='Enter City Name', style={'width': '180px', 'padding': '8px', 'borderRadius': '6px', 'border': 'none', 'marginRight': '10px', 'backgroundColor': '#1F2A44', 'color': '#FFF', 'fontSize': '14px'}),
            html.Button("Choose Location", id='update-button', n_clicks=0, style={'padding': '4px 8px', 'borderRadius': '6px', 'background': 'linear-gradient(90deg, #1E90FF, #00CED1)', 'border': 'none', 'color': '#FFF', 'fontSize': '14px', 'cursor': 'pointer'}),
            html.Div(id='error-message', style={'color': '#FF4500', 'marginTop': '8px', 'fontSize': '12px'})
        ], style={'textAlign': 'center', 'marginBottom': '30px'}),
        html.Div(id='kpi-container', style={'display': 'flex', 'justifyContent': 'space-between', 'marginBottom': '30px', 'flexWrap': 'wrap'}),
        html.Div([
            dcc.Graph(id='temp-feels-area', style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top'}),
            dcc.Graph(id='humidity-cloud-heatmap', style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top'}),
            dcc.Graph(id='rain-gauge', style={'width': '24%', 'display': 'inline-block', 'verticalAlign': 'top'}),
            dcc.Graph(id='pop-gauge', style={'width': '24%', 'display': 'inline-block', 'verticalAlign': 'top'}),
            dcc.Graph(id='wind-rose', style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),
            dcc.Graph(id='visibility-dew-bar', style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top'}),
            dcc.Graph(id='uvi-gdd-bar', style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top'}),
        ], style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'center'}),
        dcc.Interval(id='interval-update', interval=600000, n_intervals=0)
    ]
)

# OPC UA Server
def stop_server(signalnum, frame):
    global server_running
    server_running = False

def run_opc_server():
    server = Server()
    server.set_endpoint(OPC_SERVER_URL)
    server.set_server_name("Weather OPC UA Server")
    server.set_security_policy([ua.SecurityPolicyType.NoSecurity])

    uri = "http://example.org/weather"
    idx = server.register_namespace(uri)
    objects = server.get_objects_node()
    weather_obj = objects.add_object(idx, "WeatherData")
    london_obj = weather_obj.add_object(idx, "London")

    temp = london_obj.add_variable(idx, "Temperature", 20.0)
    feels_like = london_obj.add_variable(idx, "FeelsLike", 22.0)
    humidity = london_obj.add_variable(idx, "Humidity", 60.0)
    wind_speed = london_obj.add_variable(idx, "WindSpeed", 5.0)
    wind_dir = london_obj.add_variable(idx, "WindDirection", 180.0)
    rain = london_obj.add_variable(idx, "Rainfall", 0.0)
    clouds = london_obj.add_variable(idx, "Cloudiness", 50.0)
    visibility = london_obj.add_variable(idx, "Visibility", 10000.0)
    pop = london_obj.add_variable(idx, "PrecipitationProbability", 0.3)

    server.start()
    logger.info(f"OPC UA Server started at {OPC_SERVER_URL}")

    signal.signal(signal.SIGINT, stop_server)
    signal.signal(signal.SIGTERM, stop_server)

    try:
        while server_running:
            temp.set_value(np.random.uniform(15, 25))
            feels_like.set_value(temp.get_value() + np.random.uniform(-2, 2))
            humidity.set_value(np.random.uniform(40, 80))
            wind_speed.set_value(np.random.uniform(0, 10))
            wind_dir.set_value(np.random.uniform(0, 360))
            rain.set_value(np.random.uniform(0, 5))
            clouds.set_value(np.random.uniform(0, 100))
            visibility.set_value(np.random.uniform(5000, 15000))
            pop.set_value(np.random.uniform(0, 1))
            time.sleep(60)
    except Exception as e:
        logger.error(f"Server error: {e}")
    finally:
        server.stop()
        logger.info("OPC UA Server stopped")

# OPC UA Client Subscription Handler
class SubHandler(object):
    def datachange_notification(self, node, val, data):
        try:
            node_name = node.get_browse_name().Name
        except TimeoutError:
            logger.error(f"Timeout retrieving BrowseName for node {node.nodeid}")
            return
        timestamp = int(datetime.datetime.now().timestamp())
        data_point = {
            'dt': timestamp,
            'main': {},
            'wind': {},
            'clouds': {'all': 0},
            'rain': {'3h': 0},
            'visibility': 10000,
            'pop': 0,
            'source': 'opcua'  # Track data source
        }

        if node_name == "Temperature":
            data_point['main']['temp'] = val
        elif node_name == "FeelsLike":
            data_point['main']['feels_like'] = val
            data_point['main']['temp_min'] = val - 2
            data_point['main']['temp_max'] = val + 2
        elif node_name == "Humidity":
            data_point['main']['humidity'] = val
        elif node_name == "WindSpeed":
            data_point['wind']['speed'] = val
        elif node_name == "WindDirection":
            data_point['wind']['deg'] = val
        elif node_name == "Rainfall":
            data_point['rain']['3h'] = val
        elif node_name == "Cloudiness":
            data_point['clouds']['all'] = val
        elif node_name == "Visibility":
            data_point['visibility'] = val
        elif node_name == "PrecipitationProbability":
            data_point['pop'] = val

        weather_data.append(data_point)
        logger.debug(f"Data point added from OPC UA: {data_point}")

# Start OPC UA Server
Thread(target=run_opc_server, daemon=True).start()
time.sleep(2)

# OPC UA Client Setup
try:
    client = Client(OPC_SERVER_URL, timeout=10)
    client.connect()
    root = client.get_root_node()
    weather_obj = root.get_child(["0:Objects", "2:WeatherData", "2:London"])
    nodes = {
        "Temperature": weather_obj.get_child("2:Temperature"),
        "FeelsLike": weather_obj.get_child("2:FeelsLike"),
        "Humidity": weather_obj.get_child("2:Humidity"),
        "WindSpeed": weather_obj.get_child("2:WindSpeed"),
        "WindDirection": weather_obj.get_child("2:WindDirection"),
        "Rainfall": weather_obj.get_child("2:Rainfall"),
        "Cloudiness": weather_obj.get_child("2:Cloudiness"),
        "Visibility": weather_obj.get_child("2:Visibility"),
        "PrecipitationProbability": weather_obj.get_child("2:PrecipitationProbability")
    }

    handler = SubHandler()
    sub = client.create_subscription(60000, handler)
    sub.subscribe_data_change(list(nodes.values()))
    logger.info("OPC UA Client connected and subscribed")
except Exception as e:
    logger.error(f"Client connection error: {e}")

# Fallback to OpenWeatherMap
def fetch_weather_data(city):
    API_KEY = "0dcac01ceb5b8e273abc1095c5910a20"
    BASE_URL = "https://api.openweathermap.org/data/2.5/forecast"
    params = {"q": city, "appid": API_KEY, "units": "metric"}
    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        if response.status_code == 200:
            data = response.json()
            for item in data['list']:
                item['source'] = 'openweathermap'  # Track data source
            logger.debug(f"Fetched OpenWeatherMap data for {city}")
            return data
        return None
    except requests.RequestException:
        return None

# Callback for updating the dashboard
@app.callback(
    [Output('temp-feels-area', 'figure'),
     Output('humidity-cloud-heatmap', 'figure'),
     Output('rain-gauge', 'figure'),
     Output('pop-gauge', 'figure'),
     Output('wind-rose', 'figure'),
     Output('visibility-dew-bar', 'figure'),
     Output('uvi-gdd-bar', 'figure'),
     Output('kpi-container', 'children'),
     Output('error-message', 'children')],
    [Input('interval-update', 'n_intervals'),
     Input('update-button', 'n_clicks')],
    [dash.dependencies.State('city-input', 'value')]
)
def update_dashboard(n_intervals, n_clicks, city):
    if not weather_data:
        data = fetch_weather_data(city)
        if not data or 'list' not in data:
            return (dash.no_update,) * 9 + (f"Error: Unable to fetch data for '{city}'.",)
        weather_data.extend(data['list'])

    df = pd.DataFrame(weather_data)
    df['datetime'] = pd.to_datetime(df['dt'], unit='s')
    now = datetime.datetime.now()
    df = df[(df['datetime'] >= now) & (df['datetime'] <= now + datetime.timedelta(days=5))]

    if df.empty:
        return (dash.no_update,) * 9 + (f"No forecast data for '{city}' for the next 5 days.",)

    # Extract and process metrics
    df['temp'] = df['main'].apply(lambda x: x.get('temp', 0))
    df['feels_like'] = df['main'].apply(lambda x: x.get('feels_like', 0))
    df['temp_min'] = df['main'].apply(lambda x: x.get('temp_min', 0))
    df['temp_max'] = df['main'].apply(lambda x: x.get('temp_max', 0))
    df['humidity'] = df['main'].apply(lambda x: x.get('humidity', 0))
    df['clouds'] = df['clouds'].apply(lambda x: x['all'])
    df['rain'] = df['rain'].apply(lambda x: x.get('3h', 0) if isinstance(x, dict) else 0)
    df['wind_speed'] = df['wind'].apply(lambda x: x.get('speed', 0))
    df['wind_dir'] = df['wind'].apply(lambda x: x.get('deg', 0))
    df['wind_gust'] = df['wind'].apply(lambda x: x.get('gust', 0))
    df['visibility'] = df['visibility'] / 1000
    df['pop'] = df['pop'] * 100

    # Calculated metrics
    df['uvi'] = 10 * (1 - df['clouds'] / 100)
    df['gdd'] = np.maximum((df['temp_max'] + df['temp_min']) / 2 - 10, 0)
    df['wind_chill'] = np.where(
        (df['temp'] <= 10) & (df['wind_speed'] > 3),
        13.12 + 0.6215 * df['temp'] - 11.37 * (df['wind_speed'] ** 0.16) + 0.3965 * df['temp'] * (df['wind_speed'] ** 0.16),
        df['temp']
    )

    df['rain'] = df['rain'].replace(0, 0.001)
    df['visibility'] = df['visibility'].replace(0, 0.001)
    df['uvi'] = df['uvi'].replace(0, 0.001)
    df['gdd'] = df['gdd'].replace(0, 0.001)

    # Log data source
    sources = df['source'].unique()
    logger.info(f"Data sources in use: {sources}")

    # 1. Temperature & Feels Like
    fig_temp_feels = go.Figure()
    fig_temp_feels.add_trace(go.Scatter(x=df['datetime'], y=df['temp'], mode='lines', stackgroup='one', line=dict(width=0), fillcolor='rgba(0,191,255,0.5)', name='Temperature (°C)'))
    fig_temp_feels.add_trace(go.Scatter(x=df['datetime'], y=df['feels_like'], mode='lines', stackgroup='one', line=dict(width=0), fillcolor='rgba(255,105,180,0.5)', name='Feels Like (°C)'))
    fig_temp_feels.update_layout(
        title="Temperature & Feels Like (Next 5 Days)", template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font=dict(color='#FFF'), showlegend=True, xaxis_title="Time", yaxis_title="°C"
    )

    # 2. Humidity & Cloudiness
    fig_humidity_cloud = go.Figure(data=go.Heatmap(
        z=[df['humidity'], df['clouds']],
        x=df['datetime'],
        y=['Humidity (%)', 'Cloudiness (%)'],
        colorscale='Viridis',
        colorbar=dict(title="%")
    ))
    fig_humidity_cloud.update_layout(
        title="Humidity & Cloudiness Heatmap", template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font=dict(color='#FFF')
    )

    # 3. Rainfall
    fig_rain = go.Figure(go.Indicator(
        mode="gauge+number",
        value=df['rain'].mean(),
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "Avg Rainfall (mm/3h)"},
        gauge={
            'axis': {'range': [0, 10]},
            'bar': {'color': "#00BFFF"},
            'steps': [{'range': [0, 2], 'color': "lightgray"}, {'range': [2, 5], 'color': "gray"}, {'range': [5, 10], 'color': "darkgray"}],
            'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 8}
        }
    ))
    fig_rain.update_layout(
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font=dict(color='#FFF')
    )

    # 4. Probability of Precipitation
    fig_pop = go.Figure(go.Indicator(
        mode="gauge+number",
        value=df['pop'].mean(),
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "Avg Precipitation Probability (%)"},
        gauge={
            'axis': {'range': [0, 100]},
            'bar': {'color': "#00CED1"},
            'steps': [{'range': [0, 30], 'color': "lightgray"}, {'range': [30, 70], 'color': "gray"}, {'range': [70, 100], 'color': "darkgray"}],
            'threshold': {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 90}
        }
    ))
    fig_pop.update_layout(
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font=dict(color='#FFF')
    )

    # 5. Wind Rose
    wind_bins = pd.cut(df['wind_dir'], bins=np.arange(0, 361, 45), right=False)
    wind_counts = wind_bins.value_counts().sort_index()
    theta = [f"{int(b.left)}-{int(b.right)}°" for b in wind_counts.index]
    fig_wind_rose = go.Figure(go.Barpolar(
        r=wind_counts.values,
        theta=[22.5 + i * 45 for i in range(len(theta))],
        width=[45] * len(theta),
        marker_color='#00BFFF',
        marker_line_color="white",
        marker_line_width=1,
        opacity=0.8
    ))
    fig_wind_rose.update_layout(
        title="Wind Direction Distribution", template='plotly_dark', polar=dict(radialaxis=dict(visible=True, range=[0, max(wind_counts.values) * 1.2]), angularaxis=dict(direction="clockwise")),
        plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font=dict(color='#FFF')
    )

    """# 6. Visibility & Wind Chill
    fig_vis_dew = go.Figure()
    fig_vis_dew.add_trace(go.Bar(x=df['datetime'], y=df['visibility'], name='Visibility (km)', marker_color='#00CED1'))
    fig_vis_dew.add_trace(go.Bar(x=df['datetime'], y=df['wind_chill'], name='Wind Chill (°C)', marker_color='#FF69B4'))
    fig_vis_dew.update_layout(
        title="Visibility & Wind Chill", template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font=dict(color='#FFF'), barmode='group', xaxis_title="Time", yaxis_title="Value",
         # Legend settings
        legend=dict(
            orientation="h",  # Horizontal orientation
            x=0.5,  # Center horizontally
            y=0.95,  # Place above the title (adjust as needed)
            xanchor='center',  # Anchor the legend's x position at its center
            yanchor='bottom',  # Anchor the legend's y position at its bottom
            font=dict(size=12, color='#FFF'),  # Match font color with the theme
            bgcolor='rgba(0,0,0,0)',  # Transparent background for the legend
            bordercolor='rgba(0,0,0,0)',  # No border
        )
    )"""

    # 6. Visibility & Wind Chill
    fig_vis_dew = go.Figure()
    fig_vis_dew.add_trace(go.Bar(
        x=df['datetime'], y=df['visibility'], marker_color='#FFD700', opacity=0.7,
        name='Visibility (km)'  # Ensure the trace has a name for the legend
    ))
    fig_vis_dew.add_trace(go.Scatter(
        x=df['datetime'], y=df['wind_chill'], mode='lines', line=dict(color='#FF69B4', width=1),
        name='Wind Chill (°C)', yaxis='y2'
    ))
    fig_vis_dew.update_layout(
        title=dict(text="Visibility & Wind Chill (5 Days)", font=dict(color='#00BFFF', size=16, weight='bold'), y=1.0, x=0.5, xanchor='center', yanchor='top'),
        template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='#FFF'), yaxis_title="Visibility (km)",
        yaxis2=dict(overlaying='y', side='right', title="Wind Chill (°C)", showgrid=False),
        xaxis=dict(showgrid=False), yaxis=dict(showgrid=False),
        margin=dict(t=40), height=250,
        # Legend settings
        legend=dict(
            orientation="h",  # Horizontal orientation
            x=0.5,  # Center horizontally
            y=0.95,  # Place above the title (adjust as needed)
            xanchor='center',  # Anchor the legend's x position at its center
            yanchor='bottom',  # Anchor the legend's y position at its bottom
            font=dict(size=12, color='#FFF'),  # Match font color with the theme
            bgcolor='rgba(0,0,0,0)',  # Transparent background for the legend
            bordercolor='rgba(0,0,0,0)',  # No border
        )
    )

    # 7. UVI & GDD
    fig_uvi_gdd = go.Figure()
    fig_uvi_gdd.add_trace(go.Bar(x=df['datetime'], y=df['uvi'], name='UV Index', marker_color='#FFD700'))
    fig_uvi_gdd.add_trace(go.Bar(x=df['datetime'], y=df['gdd'], name='Growing Degree Days', marker_color='#32CD32'))
    fig_uvi_gdd.update_layout(
        title="UV Index & Growing Degree Days", template='plotly_dark', plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font=dict(color='#FFF'), barmode='group', xaxis_title="Time", yaxis_title="Value"
    )

    # KPIs
    kpis = [
        html.Div(f"Avg Temp: {df['temp'].mean():.1f}°C", style={'background': 'linear-gradient(90deg, #00FFFF, #4682B4)', 'padding': '5px', 'borderRadius': '5px', 'width': 'auto', 'textAlign': 'center', 'margin': '5px', 'fontSize': '12px'}),
        html.Div(f"Avg Humidity: {df['humidity'].mean():.1f}%", style={'background': 'linear-gradient(90deg, #00CED1, #4682B4)', 'padding': '5px', 'borderRadius': '5px', 'width': 'auto', 'textAlign': 'center', 'margin': '5px', 'fontSize': '12px'}),
        html.Div(f"Total Rain: {df['rain'].sum():.1f}mm", style={'background': 'linear-gradient(90deg, #1E90FF, #00CED1)', 'padding': '5px', 'borderRadius': '5px', 'width': 'auto', 'textAlign': 'center', 'margin': '5px', 'fontSize': '12px'})
    ]

    return fig_temp_feels, fig_humidity_cloud, fig_rain, fig_pop, fig_wind_rose, fig_vis_dew, fig_uvi_gdd, kpis, ""

if __name__ == '__main__':
    try:
        app.run_server(debug=True, host='127.0.0.1', port=8050)
    except KeyboardInterrupt:
        logger.info("Shutting down gracefully...")
        server_running = False

INFO:__main__:OPC UA Server started at opc.tcp://localhost:4844
Exception in thread Thread-4811 (run_opc_server):
Traceback (most recent call last):
  File "c:\Users\User\miniconda3\envs\py\Lib\threading.py", line 1041, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "c:\Users\User\miniconda3\envs\py\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "c:\Users\User\miniconda3\envs\py\Lib\threading.py", line 992, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\User\AppData\Local\Temp\ipykernel_3220\2469594066.py", line 103, in run_opc_server
    signal.signal(signal.SIGINT, stop_server)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\miniconda3\envs\py\Lib\signal.py", line 58, in signal
    handler = _signal.signal(_enum_to_int(signalnum), _enum_to_int(handler))
ValueError: signal only works in main thread

INFO:__main__:Data sources in use: ['openweathermap']
INFO:__main__:Data sources in use: ['openweathermap']
INFO:__main__:Data sources in use: ['openweathermap']
INFO:__main__:Data sources in use: ['openweathermap']
ERROR:__main__:Timeout retrieving BrowseName for node NumericNodeId(ns=2;i=3)
ERROR:__main__:Timeout retrieving BrowseName for node NumericNodeId(ns=2;i=4)
ERROR:__main__:Timeout retrieving BrowseName for node NumericNodeId(ns=2;i=5)
ERROR:__main__:Timeout retrieving BrowseName for node NumericNodeId(ns=2;i=6)
ERROR:__main__:Timeout retrieving BrowseName for node NumericNodeId(ns=2;i=7)
ERROR:__main__:Timeout retrieving BrowseName for node NumericNodeId(ns=2;i=8)
ERROR:__main__:Timeout retrieving BrowseName for node NumericNodeId(ns=2;i=9)
ERROR:__main__:Timeout retrieving BrowseName for node NumericNodeId(ns=2;i=10)
ERROR:__main__:Timeout retrieving BrowseName for node NumericNodeId(ns=2;i=11)
